In [1]:
from pymongo import MongoClient
import pandas as pd

In [2]:
def connectCollection(database, collection):
    client = MongoClient()
    db = client[database]
    coll = db[collection]
    return db, coll


In [3]:
db, coll = connectCollection('companies','companies')

In [4]:
#rest = list(coll.find({'founded_year':{'$gt':2009}}))
new_companies = list(coll.find({'$and':
                           [ { "offices.latitude": {"$exists": True,"$ne":None} },
                            { "offices.longitude": {"$exists": True,"$ne":None} },
                            {'founded_year': {'$gt': 2009 }},
                            {'deadpooled_year':None},
                            {'$or':[{'category_code': 'advertising' },
                                    {'category_code': 'mobile' },
                                    {'category_code': 'web' },
                                    {'category_code': 'games_video' }]}]}, 
                          {'name':1,'founded_year': 1, '_id': 0,'offices.latitude':1, 
                           'offices.longitude':1, 'category_code':1, 'location':1 }))

In [5]:
print(len(new_companies))

29


In [6]:
new_companies[0]

{'category_code': 'web',
 'founded_year': 2011,
 'name': 'Mokitown',
 'offices': [{'latitude': 37.09024, 'longitude': -95.712891}]}

In [7]:
df = pd.DataFrame(new_companies)
df

,category_code,founded_year,name,offices
0,web,2011,Mokitown,"[{'longitude': -95.712891, 'latitude': 37.09024}]"
1,mobile,2010,GENWI,"[{'longitude': -111.9035, 'latitude': 33.8171}]"
2,web,2013,Fixya,"[{'longitude': -122.323895, 'latitude': 37.566..."
3,games_video,2011,Social Gaming Network,"[{'longitude': -122.161523, 'latitude': 37.446..."
4,mobile,2012,Skydeck,"[{'longitude': -122.3254705, 'latitude': 37.56..."
5,mobile,2012,Jumbuck Entertainment,"[{'longitude': 144.962991, 'latitude': -37.817..."
6,web,2010,Pixsta,"[{'longitude': -0.208979, 'latitude': 51.51819}]"
7,games_video,2011,Fuzz,"[{'longitude': -122.387799, 'latitude': 37.760..."
8,web,2010,CollegeConvo,"[{'longitude': -74.294395, 'latitude': 40.6502..."
9,web,2011,Ziippi,"[{'longitude': -122.1612868, 'latitude': 37.44..."


In [8]:
#funcion para sacar las coordinadas
def getLocation(company):
    longitude = company['offices'][0]['longitude']
    latitude = company['offices'][0]['latitude']
    coordinates = [float(longitude), float(latitude)]
    
    return coordinates

In [9]:
getLocation(new_companies[0])

[-95.712891, 37.09024]

In [10]:
coords=[{'type': 'Point', 'coordinates':getLocation(comp)}for comp in new_companies]

In [11]:
#print(coords)

In [12]:
df['office_coords']=coords
df.head()

,category_code,founded_year,name,offices,office_coords
0,web,2011,Mokitown,"[{'longitude': -95.712891, 'latitude': 37.09024}]","{'type': 'Point', 'coordinates': [-95.712891, ..."
1,mobile,2010,GENWI,"[{'longitude': -111.9035, 'latitude': 33.8171}]","{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,web,2013,Fixya,"[{'longitude': -122.323895, 'latitude': 37.566...","{'type': 'Point', 'coordinates': [-122.323895,..."
3,games_video,2011,Social Gaming Network,"[{'longitude': -122.161523, 'latitude': 37.446...","{'type': 'Point', 'coordinates': [-122.161523,..."
4,mobile,2012,Skydeck,"[{'longitude': -122.3254705, 'latitude': 37.56...","{'type': 'Point', 'coordinates': [-122.3254705..."


In [13]:
df=df[['category_code','founded_year','name','office_coords']]

In [14]:
df.head()

,category_code,founded_year,name,office_coords
0,web,2011,Mokitown,"{'type': 'Point', 'coordinates': [-95.712891, ..."
1,mobile,2010,GENWI,"{'type': 'Point', 'coordinates': [-111.9035, 3..."
2,web,2013,Fixya,"{'type': 'Point', 'coordinates': [-122.323895,..."
3,games_video,2011,Social Gaming Network,"{'type': 'Point', 'coordinates': [-122.161523,..."
4,mobile,2012,Skydeck,"{'type': 'Point', 'coordinates': [-122.3254705..."


In [15]:
df.to_json('df_new_compannies.json', orient='records')

In [16]:
old_companies = list(coll.find({'$and':
                           [ { "offices.latitude": {"$exists": True,"$ne":None} },
                            { "offices.longitude": {"$exists": True,"$ne":None} },
                            {'founded_year': {'$lt': 2009 }},
                            {'deadpooled_year':None}]}, 
                          {'name':1,'founded_year': 1, '_id': 0,'offices.latitude':1, 
                           'offices.longitude':1, 'category_code':1, 'location':1 }))

In [17]:
len(old_companies)

6578

In [18]:
df_oldcomp = pd.DataFrame(old_companies)
df_oldcomp.head()

,category_code,founded_year,name,offices
0,web,2001,TechnologyGuide,"[{'longitude': -84.351532, 'latitude': 39.06685}]"
1,news,2004,Digg,"[{'longitude': -122.394523, 'latitude': 37.764..."
2,social,2004,Facebook,"[{'longitude': -122.151801, 'latitude': 37.416..."
3,web,1999,Postini,"[{'longitude': -122.247573, 'latitude': 37.506..."
4,web,2006,Geni,"[{'longitude': -118.393064, 'latitude': 34.090..."


In [19]:
old_coords=[{'type': 'Point', 'coordinates':getLocation(comp)}for comp in old_companies]

In [20]:
df_oldcomp['office_coords']=old_coords

In [21]:
df_oldcomp.head()

,category_code,founded_year,name,offices,office_coords
0,web,2001,TechnologyGuide,"[{'longitude': -84.351532, 'latitude': 39.06685}]","{'type': 'Point', 'coordinates': [-84.351532, ..."
1,news,2004,Digg,"[{'longitude': -122.394523, 'latitude': 37.764...","{'type': 'Point', 'coordinates': [-122.394523,..."
2,social,2004,Facebook,"[{'longitude': -122.151801, 'latitude': 37.416...","{'type': 'Point', 'coordinates': [-122.151801,..."
3,web,1999,Postini,"[{'longitude': -122.247573, 'latitude': 37.506...","{'type': 'Point', 'coordinates': [-122.247573,..."
4,web,2006,Geni,"[{'longitude': -118.393064, 'latitude': 34.090...","{'type': 'Point', 'coordinates': [-118.393064,..."


In [22]:
df_oldcomp=df_oldcomp[['category_code','founded_year','name','office_coords']]

In [23]:
df_oldcomp.to_json('df_old_companies.json', orient='records')

In [24]:
#esto una vez hemos añadido las coordenadas, para ver la distancia.

In [25]:
db, coll = connectCollection('companies','old_companies')

In [34]:
old_comp_paloalto = coll.find(
    {"office_coords":
     {"$near":
      {"$geometry":
       {"type":"Point",
        "coordinates":[-122.1722525,37.4374154]
       },
       "$minDistance":1000,
       "$maxDistance":6000
      }
     }
    }
)
old_comp_paloalto=list(old_comp_paloalto)

In [27]:
print(len(old_comp_paloalto))

74


In [28]:
import folium

In [35]:
map_city=folium.Map(location=[37.4374154,-122.1722525], zoom_start=12)
for c in old_comp_paloalto:
    folium.Marker(c['office_coords']['coordinates'][::-1],
                    radius=2,
                    icon=folium.Icon(icon='cloud',color='red'), 
                   ).add_to(map_city)



In [36]:
map_city

